In [49]:
import warnings
warnings.filterwarnings('ignore') # Wow nice.

In [50]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import img_to_array, ImageDataGenerator
from tensorflow.keras import backend as K

import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg') # Non-interactive

import pandas as pd

from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from sklearn.feature_extraction.image import extract_patches_2d
from sklearn.model_selection import train_test_split

import numpy as np
from imutils import paths
import cv2
import os

In [51]:
class SmallVGGNet(object):
    @staticmethod
    def build(width, height, num_classes, depth=3): # Depth is 3 cuz' RGB color-space
        model = Sequential()
        input_shape = (height, width, depth)
        chan_dim = -1 # Index of the channel dimension (in order to apply BN)
        
        # First layers set ((Conv => ReLU => BN) * 2 => POOL => DO)
        # 32 filters
        model.add(Conv2D(32, (3, 3), padding='same', input_shape=input_shape))
        model.add(Activation('relu'))
        model.add(BatchNormalization(axis=chan_dim))
        model.add(Conv2D(32, (3, 3), padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization(axis=chan_dim))
        model.add(MaxPooling2D(pool_size=(2, 2))) # Implicit stride of 2x2
        model.add(Dropout(0.25))
        
        # Second layers set ((Conv => ReLU => BN) * 2 => POOL => DO)
        # 64 filters
        model.add(Conv2D(64, (3, 3), padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization(axis=chan_dim))
        model.add(Conv2D(64, (3, 3), padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization(axis=chan_dim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        
        # FC layers set (FC => ReLU)
        model.add(Flatten())
        model.add(Dense(512))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
        
        # Softmax Classifier
        model.add(Dense(num_classes)) # Determined by the number of classes
        model.add(Activation('softmax'))
        
        print(model.summary())
        
        return model

## Network architecture

In [52]:
net = SmallVGGNet()
model = net.build(32, 32, num_classes=3) # Using 3 classes Dog, Cat and Panda

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_12 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_v1_10 (B (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_13 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_v1_11 (B (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 16, 16, 32)        0         
__________

## Loading, processing and splitting the dataset

In [53]:
print('# Loading Animals dataset...')
# trainX = training set datapoints, trainY = training set labels
# Same of testing set
image_paths = list(paths.list_images('animals'))
print('Samples:', image_paths[0: 5])
print('# Animals dataset loaded!')

# Loading Animals dataset...
Samples: ['animals/dogs/dogs_00977.jpg', 'animals/dogs/dogs_00963.jpg', 'animals/dogs/dogs_00793.jpg', 'animals/dogs/dogs_00787.jpg', 'animals/dogs/dogs_00778.jpg']
# Animals dataset loaded!


In [55]:
# Resize images and get labels
images = []
labels = []
for path in image_paths:
    # Looad the image
    image = cv2.imread(path)
    # Resize the image
    image = cv2.resize(image, (32, 32), interpolation=cv2.INTER_AREA)
    # Extract a random crop from the image
    image = extract_patches_2d(image, (32, 32), max_patches=1)[0]
    # Transform image to array
    image = img_to_array(image, data_format=None)
    # Append image and its associate label
    images.append(image)
    labels.append(path.split(os.path.sep)[-2])

# Convert the dataset to numpy array
images = np.array(images)

In [ ]:
# Scale the pixel intensity values to the range [0, 1]
def normalize(images):
    return images.astype('float') / 255.0

images = normalize(images)

In [ ]:
# Split the data into training and testing sets 75/25
(trainX, testX, trainY, testY) = train_test_split(images, labels, test_size=0.25, random_state=42)

In [ ]:
# Convert the labels from integers to vectors
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.fit_transform(testY)
print('After converting:', trainY)

In [ ]:
# Set labels names for easier reading
label_names = ['Dog', 'Cat', 'Panda']

In [ ]:
# Handle data augmentation
daug = ImageDataGenerator(rotation_range=10, width_shift_range=0.1, height_shift_range=0.1,
                         shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')

## Train the network

In [ ]:
# Define the number of epochs (times dataset has been seen)
num_epochs = 50
# Set the optimizer (here using Stochastic Gradient Descent with a learning rate of X, epochs-based decay and a momentum)
# Decay slowly reduces the learning rate to reduce overfitting and get higher classification accc
sgd_opt = SGD(lr=0.0005, decay=0.01 / num_epochs, momentum=0.9, nesterov=True)
# Compile the model
print('# Compiling the model...')
model.compile(loss='categorical_crossentropy', optimizer=sgd_opt, metrics=['accuracy'])
# Train the network!
print('# Training the network...')
M = model.fit_generator(daug.flow(trainX, trainY, batch_size=64), validation_data=(testX, testY),
                        epochs=num_epochs, steps_per_epoch=len(trainX), verbose=1)

## Evalute the network

In [ ]:
# Evalutate the network
print('# Trained the network! Evaluating...')
preds = model.predict(testX, batch_size=64)
print(classification_report(testY.argmax(axis=1),
preds.argmax(axis=1), target_names=label_names))

%matplotlib inline
# Plot history to check for overfitting
plt.style.use('ggplot')
plt.figure(figsize=(10, 10))
plt.plot(np.arange(0, num_epochs), M.history['loss'], label='train_loss')
plt.plot(np.arange(0, num_epochs), M.history['val_loss'], label='val_loss')
plt.plot(np.arange(0, num_epochs), M.history['acc'], label='train_accuracy')
plt.plot(np.arange(0, num_epochs), M.history['val_acc'], label='val_accuracy')

plt.title('Training Loss and Accuracy')
plt.ylabel('Loss/Accuracy')
plt.xlabel('Epoch #')
plt.legend()

plt.show()

## Notes:

- 1st attempt: [lr: 0.005, 50 epochs, epoch-based decay rate] : avg 75% acc